# ENSO

This notebook provides an overview of running the modes of variability module. More information can be found in the [README](https://github.com/PCMDI/pcmdi_metrics/tree/master/pcmdi_metrics/variability_mode). Example parameter files are located in the [PMP sample setups](https://github.com/PCMDI/pcmdi_metrics/tree/master/sample_setups/pcmdi_parameter_files/variability_modes). 

It is expected that the user has run the Demo_0_download_data notebook to obtain the sample data and generate parameter files. This first cell loads the demo directory choices made in Demo_0_download_data.

## Download demo data

In [ ]:
# Lets get the file containing the data needed for this demo
import requests
r = requests.get("https://pcmdiweb.llnl.gov/pss/pmpdata/pmp_enso_tutorial_files.txt")
with open("enso_data_files.txt","wb") as f:
    f.write(r.content)

In [ ]:
# This is where you will be downloading the sample_data
demo_data_directory = "enso_data"
# this line is where your output will be stored
demo_output_directory = "enso_output"

In [ ]:
# Let's download the files
import cdat_info
try:
    cdat_info.download_sample_data_files("enso_data_files.txt", demo_data_directory)
    print("All files downloaded")
except:
    print("Download failed")

In [ ]:
from download_sample_data import generate_parameter_files
filenames=["basic_enso_param.py.in"]
generate_parameter_files(demo_data_directory, demo_output_directory, filenames=filenames)

## Environment

[ENSO Metrics package](https://github.com/CLIVAR-PRP/ENSO_metrics) installation needed

In [ ]:
"""
git clone https://github.com/CLIVAR-PRP/ENSO_metrics.git
cd ENSO_metrics
python setup.py install
"""

## Usage

ENSO can be run on the command line or with using a parameter file

For help, type:  
```
enso_driver.py --help
```

In [ ]:
%%bash
enso_driver.py --help

## Basic example

In [2]:
%%bash
enso_driver.py -p basic_enso_param.py

mip: cmip5
exp: historical
models: ['ACCESS1-0']
realization:  r1i1p1
mc_name: ENSO_perf
outdir: enso_output/basicTestEnso
netcdf_path: enso_output/basicTestEnso
debug: False
obs_cmor: True
obs_cmor_path: enso_data/PCMDIobs2
egg_pth: /Users/ordonez4/Library/Caches/Python-Eggs/pcmdi_metrics-v1.2.1_683_gaed8622-py3.8.egg-tmp/share/pmp
output directory for graphics:enso_output/basicTestEnso
output directory for diagnostic_results:enso_output/basicTestEnso
output directory for metrics_results:enso_output/basicTestEnso
list_variables: ['pr', 'sst', 'taux']
list_obs: ['20CR', 'AVISO-1-0', 'CERES-EBAF-4-1', 'CMAP-V1902', 'ERA-20C', 'ERA-5', 'ERA-INT', 'GPCP-2-3', 'HadISST-1-1', 'REMSS-PRW-v07r01', 'TropFlux-1-0']
PMPdriver: dict_obs readin end
models: ['ACCESS1-0']
 ----- model:  ACCESS1-0  ---------------------
PMPdriver: var loop start for model  ACCESS1-0
realization: r1i1p1
 --- run:  r1i1p1  ---
 --- var:  pr  ---
var_in_file: pr
var, areacell_in_file, realm: pr areacella atmos
path:  en

INFO::2021-05-26 14:41::pcmdi_metrics:: Results saved to a json file: /Users/ordonez4/Documents/git/pcmdi_metrics/doc/jupyter/Demo/enso_output/basicTestEnso/cmip5_historical_ENSO_perf_basicTestEnso_ACCESS1-0_r1i1p1.json
INFO::2021-05-26 14:41::pcmdi_metrics:: Results saved to a json file: /Users/ordonez4/Documents/git/pcmdi_metrics/doc/jupyter/Demo/enso_output/basicTestEnso/cmip5_historical_ENSO_perf_basicTestEnso_ACCESS1-0_r1i1p1_diveDown.json
